In [1]:
import pandas as pd
import numpy as np
import random

!pip install pptree
from pptree import *

!pip install anytree
!pip install graphviz
from anytree import Node, RenderTree

from anytree.exporter import UniqueDotExporter

In [2]:
df = pd.read_csv("data_banknote_authentication.txt",header=None)

df.describe()
#df.head()

,0,1,2,3,4
count,1372.000000,1372.000000,1372.000000,1372.000000,1372.000000
mean,0.433735,1.922353,1.397627,-1.191657,0.444606
std,2.842763,5.869047,4.310030,2.101013,0.497103
min,-7.042100,-13.773100,-5.286100,-8.548200,0.000000
25%,-1.773000,-1.708200,-1.574975,-2.413450,0.000000
50%,0.496180,2.319650,0.616630,-0.586650,0.000000
75%,2.821475,6.814625,3.179250,0.394810,1.000000
max,6.824800,12.951600,17.927400,2.449500,1.000000


In [3]:
matrix=df.corr()
print(matrix)


          0         1         2         3         4
0  1.000000  0.264026 -0.380850  0.276817 -0.724843
1  0.264026  1.000000 -0.786895 -0.526321 -0.444688
2 -0.380850 -0.786895  1.000000  0.318841  0.155883
3  0.276817 -0.526321  0.318841  1.000000 -0.023424
4 -0.724843 -0.444688  0.155883 -0.023424  1.000000


In [4]:
matrix.shape[0]

5

In [5]:
feat = []
for i in range(0,4):
  feat.append(abs(round(matrix[i][4],2)))
#feat = [matrix[0][4],matrix[1][4],matrix[2][4],matrix[3][4]]
print(feat)
dicts = {}
keys = range(4)
for i in keys:
        dicts[i] = abs(round(feat[i],2))
print(dicts)

[0.72, 0.44, 0.16, 0.02]
{0: 0.72, 1: 0.44, 2: 0.16, 3: 0.02}


In [6]:
def score_func(features):
  score = 0;
  for x in features:
    score = score + x
  return score

In [7]:
def b_b(features , preserve, max_score, sel_features, num_final,feat_par, ):
  curr_score = score_func(features)
  print("current features with score: ",features," ",curr_score)
  
  if(curr_score <= max_score):
    return max_score,sel_features

  if(len(features) == num_final):
    if(curr_score > max_score):
      max_score = curr_score
      sel_features = features.copy()
    return max_score,sel_features
  
  num_branches = num_final + 1 - len(preserve)
  removal_list = random.sample([x for x in features if x not in preserve], k=num_branches)
  print("removal_list: ",removal_list)

  for i in range(len(removal_list)):
    print("feature removed with correlation value: ",removal_list[len(removal_list)-1 - i])
    feat_pass = [x for x in features if x!= removal_list[len(removal_list)-1 - i]]
    child_feat = Node(feat_pass, parent = feat_par)
    pres_pass = preserve.copy()
    pres_pass.extend(removal_list[:len(removal_list)-1 - i])

    max_score,sel_features = b_b(feat_pass , pres_pass, max_score, sel_features, num_final,child_feat)

  return max_score,sel_features

In [8]:
def get_key(val):
    for key, value in dicts.items():
        if val == value:
            return key
 
    return "key doesn't exist"

In [10]:
feat=[0,1,2,3]
num_sel_features = 2
#max_score = score_func(features)
feat_par = Node(feat)

max_score,sel_feat=b_b(feat , [], float('-inf'), [], num_sel_features, feat_par )
print(get_key(sel_feat[0]),get_key(sel_feat[1]))
UniqueDotExporter(feat_par).to_picture("tree.png")

current features with score:  [0, 1, 2, 3]   6
removal_list:  [3, 2, 0]
feature removed with correlation value:  0
current features with score:  [1, 2, 3]   6
removal_list:  [1]
feature removed with correlation value:  1
current features with score:  [2, 3]   5
feature removed with correlation value:  2
current features with score:  [0, 1, 3]   4
feature removed with correlation value:  3
current features with score:  [0, 1, 2]   3
key doesn't exist key doesn't exist


FileNotFoundError: [Errno 2] No such file or directory: 'dot'